In [1]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import replicate
import os
from dotenv import load_dotenv

load_dotenv()

/opt/homebrew/Cellar/python@3.10/3.10.17_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
pairs_hf = [
    ["what is panda?", "hi"],
    ["what is panda?", "The giant panda is a bear species endemic to China."],
]

pairs_replicate = {
        "input_list": '[["what is panda?", "hi"], ["what is panda?", "The giant panda is a bear species endemic to China."]]'
    }

In [3]:
# Huggingface
start_time = time.time()

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "BAAI/bge-reranker-base", torch_dtype=torch.float16
).to("cpu")
model.eval()

with torch.no_grad():
    inputs = tokenizer(
        pairs_hf, padding=True, truncation=True, return_tensors="pt", max_length=512
    ).to("cpu")
    scores = (
        model(**inputs, return_dict=True)
        .logits.view(
            -1,
        )
        .float()
    )

print(f"Scores: {scores.tolist()}")
print(f"Time: {time.time() - start_time:.2f}s")


Scores: [-8.1484375, 2.40625]
Time: 4.35s


In [4]:
# Replicate
start_time = time.time()

output = replicate.run(
    "yxzwayne/bge-reranker-v2-m3:7f7c6e9d18336e2cbf07d88e9362d881d2fe4d6a9854ec1260f115cabc106a8c",
    input=pairs_replicate,
)
print(output)
print(f"Time: {time.time() - start_time:.2f}s")

[-8.1796875, 1.65234375]
Time: 21.20s
